In [1]:
from transformers import MBartForConditionalGeneration, AutoTokenizer

from huggingface_hub import notebook_login
from transformers import MBart50Tokenizer
import tqdm
import torch
import evaluate
import pandas as pd

import os

In [2]:
val = pd.read_json('/home/yush/kreol-benchmark/data/lang_data/en-cr/en-cr_dev.jsonl',lines=True)
val_inputs = list(val['input'])
val_labels = list(val['target'])

In [3]:
bleu = evaluate.load("bleu")
chrf = evaluate.load("chrf")

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device='cpu'

In [5]:
weights = os.listdir('/home/yush/kreol-benchmark/checkpoint')
weights = [x for x in weights if 'checkpoint' in x]

In [7]:
for ckpt in weights:
    checkpoint = os.path.join('/home/yush/kreol-benchmark/checkpoint',ckpt)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = MBartForConditionalGeneration.from_pretrained(checkpoint)
    model = model.to(device)
    input_tokens = tokenizer(val_inputs,max_length=128, truncation=True, padding="max_length",return_tensors='pt').to(device)
    output_tokens = model.generate(**input_tokens)
    output = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)
    print(checkpoint)
    print(chrf.compute(predictions=output,references=val_labels))
    print(bleu.compute(predictions=output,references=val_labels))
    print('----------------')
    break

/home/yush/kreol-benchmark/checkpoint/checkpoint-140000
{'score': 44.84547543879345, 'char_order': 6, 'word_order': 0, 'beta': 2}
{'bleu': 0.2206957217203022, 'precisions': [0.5968494749124854, 0.3073110285006196, 0.17582562747688243, 0.10352142554094187], 'brevity_penalty': 0.9181318826103453, 'length_ratio': 0.9213072457536013, 'translation_length': 8570, 'reference_length': 9302}
----------------


In [10]:
output

['Mo pa ti vini pou fer fas ar zot, me pou donn zot tou zot sinifikasion.',
 'Anfet, dan sa lekol la, u ti bizin pey profeser pu al lekol.',
 'Angina kapav dekrir li kuma enn dekurazman, enn lour, depresyasyon, brile, brile.',
 'Garson la ti dir li pou, me li pa ti ale.',
 'Eski Bondie dan lesiel oubien imin?',
 'Apre enn ki byen valab, proprieter la al vann tu pu aste perl.',
 'Zot pey li lor karo kok, kouma Lesegner ti donn mwa lord.',
 'Alors kan u al enn lavil ubyen enn vilaz, truv kikenn ena ase valer pu ki u vinn so invite e res ar zot ziska u ale.',
 'Ler zot ariv laba, li ti dir zot, asiz laba e priye.',
 'Li al kot bann sef pret e li dir li, komie zot pou donn mwa si mo ed zot aret Zezi?',
 'Apre enn trompet, li pou avoy so bann anz pou amenn so bann anz ki li finn swazir lor later.',
 'Dite pu anplway bann anplwaye ki U galaksi ena enn Veto.',
 'Li vagn zanfan kumadir: “Mo pe vini”.',
 'Li pa bizin bwar divin ousa labier e puvwar Lespri Sin pou vinn avek li kan li ne.',
 'Mo 